## Estimativa de compactação

In [11]:

declare @nomeDaTabela as varchar(50) = 'tItemMovimento'
declare @tabela table (
    a varchar(128),
    b varchar(128),
    c int,
    d int,
    [size_with_current_compression_setting(KB)] bigint,
    [size_with_requested_compression_setting(KB)] bigint,
    e bigint,
    f bigint
)
declare @tamanhoAtual as bigint
declare @tamanhoEstimado as bigint
declare @porcentagemDeMelhoria as bigint

insert @tabela (a, b, c, d, [size_with_current_compression_setting(KB)], [size_with_requested_compression_setting(KB)], e, f)
exec sp_estimate_data_compression_savings 'dbo', @nomeDaTabela, null, null, 'PAGE'
select @tamanhoAtual = [size_with_current_compression_setting(KB)],
       @tamanhoEstimado = [size_with_requested_compression_setting(KB)]
from @tabela

set @porcentagemDeMelhoria = 100- ((@tamanhoEstimado * 100) / @tamanhoAtual)
select @nomeDaTabela as tabela,
       @tamanhoAtual as tamanhoAtual,
       @tamanhoEstimado as tamanhoEstimado,
       @porcentagemDeMelhoria as porcentagemDeMelhoria

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.392

tabela,tamanhoAtual,tamanhoEstimado,porcentagemDeMelhoria
tItemMovimento,160704,66280,59


## Mostra as paginas que estão alocadas na tabela

In [12]:
declare @nomeDaTabela as varchar(50) = 'tItemMovimento'
select @nomeDaTabela as tabela, total_pages, used_pages, data_pages, p.data_compression_desc
from sys.allocation_units au
         join sys.partitions p
              on au.container_id = p.partition_id
where p.object_id = object_id( @nomeDaTabela )
  and au.type = 1

(1 row affected)

Total execution time: 00:00:00.008

tabela,total_pages,used_pages,data_pages,data_compression_desc
tItemMovimento,20090,20088,20086,NONE


## Tamanho atual

In [14]:
sp_spaceused 'tItemMovimento'

Commands completed successfully.

Total execution time: 00:00:00.003

name,rows,reserved,data,index_size,unused
tItemMovimento,2611100,160720 KB,160688 KB,16 KB,16 KB


## Resultado

In [18]:
sp_spaceused 'tItemMovimento'

Commands completed successfully.

Total execution time: 00:00:00.003

name,rows,reserved,data,index_size,unused
tItemMovimento,2611100,65616 KB,65552 KB,16 KB,48 KB


## Compactando os dados

In [16]:
alter table dbo.tItemMovimento
    rebuild partition = all
with  (data_compression = page )

Commands completed successfully.

Total execution time: 00:00:05.269

## Resultado compactação

In [17]:
declare @nomeDaTabela as varchar(50) = 'tItemMovimento'
select @nomeDaTabela as tabela, total_pages, used_pages, data_pages, p.data_compression_desc
from sys.allocation_units au
         join sys.partitions p
              on au.container_id = p.partition_id
where p.object_id = object_id( @nomeDaTabela )
  and au.type = 1

(1 row affected)

Total execution time: 00:00:00.001

tabela,total_pages,used_pages,data_pages,data_compression_desc
tItemMovimento,8202,8196,8194,PAGE
